In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pdfplumber

c:\Users\Akhil\projects2\anukta-rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Importing .txt document and chunking

In [24]:
def chunk_text_simple(text, max_tokens):
    # Split the text into words
    words = text.split()

    # Group words into chunks of at most max_tokens
    chunks = [
        " ".join(words[i:i + max_tokens])
        for i in range(0, len(words), max_tokens)
    ]

    return chunks

with open('hp_manual.txt', 'r', encoding='utf-8') as f:
    full_text = f.read()


In [25]:
data = chunk_text_simple(full_text, 500)

In [26]:
data

['User Guide © Copyright 2014 Hewlett-Packard Development Company, L.P. Bluetooth is a trademark owned by its proprietor and used by Hewlett-Packard Company under license. Intel is a trademark of Intel Corporation in the U.S. and other countries. Microsoft and Windows are U.S. registered trademarks of the Microsoft group of companies. The information contained herein is subject to change without notice. The only warranties for HP products and services are set forth in the express warranty statements accompanying such products and services. Nothing herein should be construed as constituting an additional warranty. HP shall not be liable for technical or editorial errors or omissions contained herein. Second Edition: October 2014 First Edition: September 2014 Document Part Number: 793368-002 Product notice This guide describes features that are common to most models. Some features may not be available on your tablet. Not all features are available in all editions of Windows. This tablet 

# Embedding

In [27]:
# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight & fast

In [28]:
embeddings = embedder.encode(data, convert_to_numpy=True)

In [29]:
embeddings

array([[-0.10406009, -0.03900221,  0.04855574, ...,  0.01949895,
         0.0844449 ,  0.02467618],
       [-0.02790189, -0.05774884, -0.11732259, ...,  0.02115861,
         0.04342958, -0.00750632],
       [-0.04818346, -0.03612443,  0.06029768, ...,  0.02051263,
         0.01128818,  0.00598023],
       ...,
       [-0.0897944 , -0.02082036,  0.04850111, ...,  0.04992981,
         0.0075749 , -0.05266475],
       [-0.05413851, -0.00484724, -0.00911877, ...,  0.08714322,
        -0.03672336, -0.06100498],
       [-0.05573836,  0.00018321,  0.05250529, ..., -0.02503051,
        -0.05501289,  0.00355061]], shape=(18, 384), dtype=float32)

# Create index

In [30]:
import numpy as np

def normalize_embeddings(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms


In [31]:
dimension = embeddings.shape[1]

In [32]:
index = faiss.IndexFlatIP(dimension)

In [33]:
normalized_embeddings = normalize_embeddings(embeddings)

In [34]:
index.add(normalized_embeddings)

# Query, Semantic Search and Generation

In [35]:
query = "How do I connect to a bluetooth network?"

In [36]:
query_embedding = embedder.encode([query], convert_to_numpy=True)
normalized_query = normalize_embeddings(query_embedding)


In [37]:
k = 1  # number of nearest neighbors
D, I = index.search(normalized_query, k)

In [38]:
data[I[0][0]]

'be equipped with one or more of the following wireless devices: ● WLAN device—Connects the tablet to wireless local area networks (commonly referred to as WiFi networks, wireless LANs, or WLANs) in corporate offices, your home, and public places such as airports, restaurants, coffee shops, hotels, and universities. In a WLAN, the mobile wireless device in your tablet communicates with a wireless router or a wireless access point. ● Bluetooth device—Creates a personal area network (PAN) to connect to other Bluetoothenabled devices such as tablets, phones, printers, headsets, speakers, and cameras. In a PAN, each device communicates directly with other devices, and devices must be relatively close together—typically within 10 meters (approximately 33 feet) of each other. For more information about wireless technology, the Internet, and networking, see the information and website links provided in Help and Support. ▲ From the Start screen, tap , type help, and then select Help and Suppor

In [39]:
import ollama

prompt = f"""
Answer the question using the context.

Question: {query}

Context: {data[I[0][0]]}
"""

response = ollama.chat(
    model='qwen3:4b',
    messages=[{'role': 'user', 'content': prompt}]
)

print(response['message']['content'])

<think>
Okay, the user is asking how to connect to a Bluetooth network. Let me look through the provided context to find the relevant information.

The context mentions Bluetooth devices creating a personal area network (PAN) to connect to other Bluetooth-enabled devices. It also talks about the Network and Sharing Center for setting up connections. However, the specific steps for Bluetooth might be in the "Connecting to a wireless network" section, but that's under WLAN. Wait, the context has a part about Bluetooth devices, but the steps given are for WLAN. Hmm.

Wait, the context says: "To use operating system controls: 1. From the Start screen, tap , type control panel, and then select Control Panel. 2. Select Network and Internet, and then select Network and Sharing Center." But the steps for connecting to a WLAN are detailed. However, the Bluetooth connection steps aren't explicitly listed here. The context does mention that Bluetooth creates a PAN, but the steps for connecting vi

In [41]:
## cleaning 
import re

raw_content = response['message']['content']

# Remove all <think>...</think> tags and their content
clean_content = re.sub(r'<think>.*?</think>', '', raw_content, flags=re.DOTALL)

clean_content = clean_content.strip()

query_response = clean_content
print(query_response)

To connect to a Bluetooth network (personal area network, or PAN), follow these steps based on the context provided:

1. **Ensure Bluetooth is enabled**: Verify that your device's Bluetooth is turned on. This is typically done through the device's settings or a dedicated Bluetooth icon in the notification area.

2. **Access Network and Sharing Center**: 
   - From the Start screen, type "Control Panel" and select it.
   - Navigate to **Network and Internet > Network and Sharing Center**.

3. **Set up a Bluetooth connection**:
   - In the Network and Sharing Center, look for options related to Bluetooth devices or PANs. The context mentions that the center allows setting up connections, so you may need to manually search for or pair devices.
   - If no Bluetooth options are visible, check the Help and Support links provided in the context for further guidance.

4. **Pair devices**: 
   - Bluetooth devices (e.g., printers, speakers) must be within 10 meters of each other. Enable Bluetoot

# Feedback

In [42]:
feedback = "Bluetooth connections can now work up till 25 metres."

# LLM Call for updating chunk

In [44]:
import ollama

prompt = f"""
chunk: {data[I[0][0]]}
----
feedback: {feedback}
----
response: {query_response}
----

You are an AI agent in a RAG workflow. The user had queried, a chunk was retrieved and you provided a response.
Now the user wants to modify the chunk with the new information.

Your task is to carefully incorporate the feedback into the chunk.

Do NOT paraphrase or rephrase the entire chunk. Instead, reframe only the necessary sentence(s) to fully integrate the feedback.

Print ONLY AND ONLY the updated chunk, exactly as it should be, without any additional comments, recommendations, or introductory text.

After updating, ensure the entire chunk is printed with the feedback fully incorporated.
For example if the chunk is: 'after that reset. The color should be green'. And we update it to dark green. Then return only this:
'after that reset. The color should be dark green.'
"""


response = ollama.chat(
    model='qwen3:4b',
    messages=[{'role': 'user', 'content': prompt}]
)

print(response['message']['content'])

<think>
Okay, let's see. The user wants me to incorporate the feedback into the chunk. The feedback says that Bluetooth connections can now work up to 25 meters. The original chunk mentions that Bluetooth devices must be within 10 meters. So I need to update that part.

Looking at the original chunk, the relevant sentence is: "devices must be relatively close together—typically within 10 meters (approximately 33 feet) of each other." The feedback changes that to 25 meters. So I should replace the 10 meters with 25 meters. Let me check if there's any other mention of Bluetooth in the chunk. The rest of the chunk is about connecting to a WLAN and using the Network and Sharing Center, so that part stays the same. Just need to adjust the Bluetooth range. Make sure to keep the rest of the text as is, only changing that specific sentence. Alright, that should do it.
</think>

be equipped with one or more of the following wireless devices: ● WLAN device—Connects the tablet to wireless local a

## Update Chunk

In [45]:
import re

raw_content = response['message']['content']

# Remove all <think>...</think> tags and their content
clean_content = re.sub(r'<think>.*?</think>', '', raw_content, flags=re.DOTALL)

clean_content = clean_content.strip()

print(clean_content)

be equipped with one or more of the following wireless devices: ● WLAN device—Connects the tablet to wireless local area networks (commonly referred to as WiFi networks, wireless LANs, or WLANs) in corporate offices, your home, and public places such as airports, restaurants, coffee shops, hotels, and universities. In a WLAN, the mobile wireless device in your tablet communicates with a wireless router or a wireless access point. ● Bluetooth device—Creates a personal area network (PAN) to connect to other Bluetoothenabled devices such as tablets, phones, printers, headsets, speakers, and cameras. In a PAN, each device communicates directly with other devices, and devices must be relatively close together—typically within 25 meters (approximately 82 feet) of each other. For more information about wireless technology, the Internet, and networking, see the information and website links provided in Help and Support. ▲ From the Start screen, tap , type help, and then select Help and Support

In [46]:
# Create embedding for updated content
new_embedding = embedder.encode([clean_content], convert_to_numpy=True)
normalized_new_embedding = normalize_embeddings(new_embedding)

# Remove the vector at index I[0][0]
index = faiss.IndexFlatIP(dimension)  # Create fresh index
mask = np.ones(len(normalized_embeddings), dtype=bool)
mask[I[0][0]] = False
filtered_embeddings = normalized_embeddings[mask]

# Add all vectors except the one we want to replace
index.add(filtered_embeddings)

# Add the new normalized embedding
index.add(normalized_new_embedding)


# Update the data list as well
data = [x for i, x in enumerate(data) if i != I[0][0]]
data.append(clean_content)

# New Query

In [47]:
query = "What is the bluetooth range?"

In [48]:
query_embedding = embedder.encode([query], convert_to_numpy=True)
normalized_query = normalize_embeddings(query_embedding)


In [50]:
k = 1  # number of nearest neighbors
D, I = index.search(normalized_query, k)


In [51]:
data[I[0][0]]

'be equipped with one or more of the following wireless devices: ● WLAN device—Connects the tablet to wireless local area networks (commonly referred to as WiFi networks, wireless LANs, or WLANs) in corporate offices, your home, and public places such as airports, restaurants, coffee shops, hotels, and universities. In a WLAN, the mobile wireless device in your tablet communicates with a wireless router or a wireless access point. ● Bluetooth device—Creates a personal area network (PAN) to connect to other Bluetoothenabled devices such as tablets, phones, printers, headsets, speakers, and cameras. In a PAN, each device communicates directly with other devices, and devices must be relatively close together—typically within 25 meters (approximately 82 feet) of each other. For more information about wireless technology, the Internet, and networking, see the information and website links provided in Help and Support. ▲ From the Start screen, tap , type help, and then select Help and Suppor

In [52]:
import ollama

prompt = f"""
Answer the question using the context.

Question: {query}

Context: {data[I[0][0]]}
"""

response = ollama.chat(
    model='qwen3:4b',
    messages=[{'role': 'user', 'content': prompt}]
)

print(response['message']['content'])

<think>
Okay, so the question is asking about the Bluetooth range. Let me look through the provided context to find the relevant information.

The context mentions a Bluetooth device which creates a personal area network (PAN). It says that in a PAN, each device communicates directly with others and must be relatively close together—typically within 25 meters (approximately 82 feet) of each other. 

So the key point here is that the Bluetooth range is about 25 meters. I should check if there's any other information that might contradict or add to this. The rest of the context talks about WLAN connections, but the Bluetooth part specifically states 25 meters. The note about being out of range for WLAN is separate, so it's not relevant here. 

Therefore, the answer should be that the Bluetooth range is typically within 25 meters or approximately 82 feet.
</think>

The Bluetooth range is typically within **25 meters (approximately 82 feet)**. This allows devices to communicate directly in